In [2]:
import pymysql
import pandas as pd

In [36]:
# 데이터베이스 접속 정보 설정
connection = pymysql.connect(
    host='ahnbi1.suwon.ac.kr',
    user='mimic4_mon_5_reader',
    password='suwondatabase',
    db='mimic4_fri_7'
)

try:
    with connection.cursor() as cursor:
        # SQL 쿼리 작성
        query = """
        WITH medication_list AS (
            SELECT DISTINCT medication, subject_id
            FROM hosp_emar
            GROUP BY medication
        ), medication_per_diagnosis AS
        (
            SELECT DISTINCT icd_code, subject_id, GROUP_CONCAT(medication) AS medication
            FROM hosp_diagnoses_icd NATURAL JOIN medication_list
            GROUP BY icd_code
            ORDER BY icd_code
        )
        SELECT DISTINCT icd_code, long_title, medication
        FROM hosp_d_icd_diagnoses NATURAL JOIN medication_per_diagnosis
        GROUP BY icd_code;
        """
        
        # 쿼리 실행 및 결과 가져오기
        cursor.execute(query)
        result = cursor.fetchall()
        
        # 결과를 데이터프레임으로 변환
        df4 = pd.DataFrame(result, columns=['icd_code', 'long_title', 'medication'])
 
        # 결과 출력
        #print(df4.head(50))
        
        # 엑셀 파일로 저장
        excel_file_path = "medication_per_diagnosis.xlsx"
        df4.to_excel(excel_file_path, index=False)  # 인덱스는 저장하지 않음
        
        print("Dataframe saved to Excel file successfully!")

except Exception as e:
    print(f"An error occurred: {e}")
    connection.rollback()  # 오류 발생 시 롤백

finally:
    # 데이터베이스 연결 닫기
    if connection:
        connection.close()

Dataframe saved to Excel file successfully!


In [35]:
print(len(df4))

9849
